In [2]:
import os
os.chdir("../")
os.chdir("FinalDataset")

In [3]:
os.getcwd()

'C:\\Users\\SRIKANTH\\Documents\\GitHub\\url_classification_dl\\FinalDataset'

In [4]:
import pandas as pd
import tensorflow as ts
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

In [109]:
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score

In [59]:
from sklearn.preprocessing import MinMaxScaler

In [24]:
m = ts.keras.metrics.CategoricalAccuracy()

In [25]:
from keras.wrappers.scikit_learn import KerasClassifier

from keras.utils import np_utils

from sklearn.model_selection import cross_val_score

from sklearn.model_selection import KFold

In [26]:
data = pd.read_csv("URL_features.csv")

In [27]:
data.drop(columns='Unnamed: 0',inplace=True)
'''
data.replace('Benign_list_big_final',0,inplace=True)
data.replace('DefacementSitesURLFiltered',1,inplace=True)
data.replace('Malware_dataset',2,inplace=True)
data.replace('phishing_dataset',3,inplace=True)
data.replace('spam_dataset',4,inplace=True)
'''
data.replace(True,1,inplace = True)
data.replace(False,0,inplace = True)
y = data["File"]
data.drop(columns = "ext",inplace = True)

In [28]:
data = data.drop(columns = "File")

In [29]:
encoder = LabelEncoder()
encoder.fit(y)
Y = encoder.transform(y)

In [60]:
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(data)
X = pd.DataFrame(X)

In [81]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [143]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding

input_dim = len(data.columns)
model = Sequential()
model.add(Dense(128, input_dim = input_dim , activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(5, activation = 'softmax'))



model.compile(loss = 'categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy',f1_m,precision_m, recall_m] )
    

In [144]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, Y, test_size=0.25, random_state=42)

In [156]:
model.fit(X_train,np_utils.to_categorical(y_train),epochs = 12,validation_split=0.3, batch_size = 128)

Epoch 1/12
679/679 [==============================] - 2s 3ms/step - loss: 0.6745 - accuracy: 0.7694 - f1_m: 0.7657 - precision_m: 0.8117 - recall_m: 0.7263 - val_loss: 0.6079 - val_accuracy: 0.7841 - val_f1_m: 0.7852 - val_precision_m: 0.8195 - val_recall_m: 0.7539
Epoch 2/12
679/679 [==============================] - 2s 3ms/step - loss: 0.5965 - accuracy: 0.7898 - f1_m: 0.7848 - precision_m: 0.8228 - recall_m: 0.7506 - val_loss: 0.5778 - val_accuracy: 0.7928 - val_f1_m: 0.7906 - val_precision_m: 0.8171 - val_recall_m: 0.7660
Epoch 3/12
679/679 [==============================] - 2s 3ms/step - loss: 0.5781 - accuracy: 0.7949 - f1_m: 0.7907 - precision_m: 0.8293 - recall_m: 0.7559 - val_loss: 0.5676 - val_accuracy: 0.8010 - val_f1_m: 0.7938 - val_precision_m: 0.8327 - val_recall_m: 0.7586
Epoch 4/12
679/679 [==============================] - 2s 3ms/step - loss: 0.5649 - accuracy: 0.8013 - f1_m: 0.7975 - precision_m: 0.8337 - recall_m: 0.7646 - val_loss: 0.5519 - val_accuracy: 0.8086 - va

In [157]:
y_pred = model.predict(X_test)

In [158]:
predicted = np.argmax(y_pred, axis=1)

In [159]:
from sklearn.metrics import classification_report
print(accuracy_score(y_test,predicted))

0.8193362682018287


In [160]:
target_names = ['Benign_list_big_final','DefacementSitesURLFiltered','Malware_dataset','phishing_dataset','spam_dataset']
print(classification_report(y_test, predicted, target_names=target_names))

                            precision    recall  f1-score   support

     Benign_list_big_final       0.77      0.88      0.82      8903
DefacementSitesURLFiltered       0.84      0.93      0.88     23970
           Malware_dataset       0.85      0.52      0.65      2864
          phishing_dataset       0.66      0.17      0.27      2493
              spam_dataset       0.79      0.58      0.67      3112

                  accuracy                           0.82     41342
                 macro avg       0.78      0.62      0.66     41342
              weighted avg       0.81      0.82      0.80     41342



In [155]:
model.save("nn1")

INFO:tensorflow:Assets written to: nn1\assets
